# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 23, 15, 10

while True:
    dt = datetime.now()
    if dt.month >= month and dt.day >= day:
        if dt.hour > hour or (dt.hour == hour and dt.minute > minute):
            break
    sleep(60)

# Parameter optimization

In [2]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['PriceChange']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 50
load = False

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'PriceChange': {"low_price_quantile": [1 * (i + 1) for i in range(20)]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['indicator_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat = stat.sort_values(['indicator_rank', 'pct_right_forecast_avg'], ascending=False)
stat.head(20)

Number of combinations is 20


  0%|          | 0/20 [00:00<?, ?it/s]

,pattern,PriceChange_low_price_quantile,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,indicator_rank
10,PriceChange,11,100.0,77.78,77.78,55.56,77.78,66.67,77.78,88.89,88.89,77.78,100.00,100.00,100.00,77.78,66.67,66.67,66.67,66.67,55.56,77.78,77.78,88.89,77.78,88.89,1.37,1.15,0.62,1.23,2.65,1.79,2.26,2.41,1.98,1.10,3.05,3.08,1.45,1.53,1.70,1.45,1.17,0.59,0.90,0.64,1.17,2.41,2.26,2.70,9,79.168750,82.518750
7,PriceChange,8,100.0,66.67,66.67,50.00,66.67,50.00,66.67,83.33,83.33,66.67,100.00,100.00,100.00,83.33,66.67,83.33,66.67,83.33,66.67,100.00,100.00,100.00,100.00,100.00,2.21,1.29,0.68,0.93,1.17,1.43,1.72,2.48,1.46,1.39,4.40,3.88,1.62,1.20,1.95,3.12,2.05,0.84,1.90,1.32,1.94,2.56,2.48,3.17,6,81.250417,67.502500
8,PriceChange,9,100.0,71.43,71.43,57.14,71.43,57.14,71.43,85.71,85.71,71.43,100.00,100.00,100.00,85.71,57.14,71.43,57.14,71.43,57.14,85.71,85.71,100.00,85.71,100.00,2.04,1.39,0.62,1.53,1.45,1.79,1.79,2.41,0.78,0.72,3.38,3.08,1.45,1.53,0.29,1.45,1.17,0.77,0.90,0.64,1.17,2.41,2.26,2.70,7,79.165417,64.157917
9,PriceChange,10,100.0,71.43,71.43,57.14,71.43,57.14,71.43,85.71,85.71,71.43,100.00,100.00,100.00,85.71,57.14,71.43,57.14,71.43,57.14,85.71,85.71,100.00,85.71,100.00,2.04,1.39,0.62,1.53,1.45,1.79,1.79,2.41,0.78,0.72,3.38,3.08,1.45,1.53,0.29,1.45,1.17,0.77,0.90,0.64,1.17,2.41,2.26,2.70,7,79.165417,64.157917
5,PriceChange,6,100.0,80.00,80.00,60.00,80.00,60.00,60.00,80.00,80.00,60.00,100.00,100.00,100.00,80.00,60.00,80.00,60.00,80.00,80.00,100.00,100.00,100.00,100.00,100.00,3.05,1.45,1.13,2.09,1.45,3.16,2.26,1.35,0.23,0.68,6.99,5.95,1.80,2.10,3.60,4.78,2.93,0.90,2.89,2.01,2.70,2.70,2.70,3.64,5,82.500000,62.500000
6,PriceChange,7,100.0,80.00,80.00,60.00,80.00,60.00,60.00,80.00,80.00,60.00,100.00,100.00,100.00,80.00,60.00,80.00,60.00,80.00,80.00,100.00,100.00,100.00,100.00,100.00,3.05,1.45,1.13,2.09,1.45,3.16,2.26,1.35,0.23,0.68,6.99,5.95,1.80,2.10,3.60,4.78,2.93,0.90,2.89,2.01,2.70,2.70,2.70,3.64,5,82.500000,62.500000
4,PriceChange,5,100.0,66.67,66.67,33.33,66.67,66.67,66.67,100.00,100.00,66.67,100.00,100.00,100.00,100.00,66.67,66.67,66.67,100.00,100.00,100.00,100.00,100.00,100.00,100.00,1.37,1.13,0.23,-0.23,0.90,3.16,2.26,1.35,0.23,0.68,1.80,1.80,1.80,2.10,3.60,6.61,6.01,6.91,3.30,2.70,2.70,4.97,2.70,2.70,3,84.723333,44.170000
3,PriceChange,4,100.0,50.00,50.00,0.00,50.00,50.00,50.00,100.00,100.00,50.00,100.00,100.00,100.00,100.00,50.00,50.00,50.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,2.60,0.49,-0.03,-0.63,-0.07,-0.03,0.25,0.96,0.21,0.17,3.95,11.35,12.74,13.84,8.04,6.67,5.44,5.22,4.84,3.84,3.37,5.01,4.91,2.72,2,77.083333,14.166667
11,PriceChange,12,100.0,72.73,72.73,54.55,72.73,54.55,63.64,81.82,81.82,72.73,90.91,90.91,81.82,63.64,54.55,54.55,54.55,63.64,54.55,72.73,63.64,81.82,63.64,72.73,1.36,1.15,0.62,1.18,1.45,1.70,1.79,1.98,1.98,1.10,3.38,3.08,1.45,1.53,1.70,1.45,0.33,0.58,0.90,0.64,1.13,2.41,2.26,2.70,11,70.457500,5.032500
12,PriceChange,13,100.0,72.73,72.73,54.55,72.73,54.55,63.64,81.82,81.82,72.73,90.91,90.91,81.82,63.64,54.55,54.55,54.55,63.64,54.55,72.73,63.64,81.82,63.64,72.73,1.36,1.15,0

# Save new config data to config file

In [13]:
from config_updater import ConfigUpdater

timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'PriceChange': {"low_price_quantile": [12]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)